In [ ]:
import sys
from PT_HGNN.data import *
from PT_HGNN.model import *
from warnings import filterwarnings
filterwarnings("ignore")

import numpy as np
import random

In [ ]:
graph = renamed_load(open("dataset/graph_Engineering.pk", 'rb'))

# train_range = {t: True for t in graph.times if t != None and t < 2015}
# valid_range = {t: True for t in graph.times if t != None and t >= 2015  and t <= 2016}
# test_range  = {t: True for t in graph.times if t != None and t > 2016}

In [ ]:
graph.node_feature['paper'].keys()
graph.get_meta_graph()
types = graph.get_meta_graph()
types

In [ ]:
targetid2sourceidlist = defaultdict( #target_type
                                    lambda: defaultdict(  #source_type
                                        lambda: defaultdict(  #relation_type
                                            lambda: defaultdict(  #target_id
                                                list #source_id list
                                                ))))

In [ ]:
targetid2degree = defaultdict( #target_type
                                lambda: defaultdict(  #source_type
                                    lambda: defaultdict(  #relation_type
                                        lambda: defaultdict(  #target_id
                                            int # each node corresponding node degree
                                            ))))

In [ ]:
for target_type in graph.edge_list:
    te = graph.edge_list[target_type]
    for source_type in te:
        tes = te[source_type]
        for relation_type in tes:
            tesr = tes[relation_type]
            for target_id in tesr:
                for source_id in tesr[target_id]:
                    targetid2sourceidlist[target_type][source_type][relation_type][target_id].append(source_id)
                targetid2degree[target_type][source_type][relation_type][target_id] = len(targetid2sourceidlist[target_type][source_type][relation_type][target_id])

In [ ]:
def calc_ppr_node(inode, target_list, target_degree, source_list, source_degree, alpha, epsilon):
    
    alpha_eps = alpha * epsilon
    
    p = {}
    r_t = {}
    r_t[inode] = alpha
    r_s = {}
    q_t = [inode]
    q_s = []
    
    t2s = True
    while len(q_t) > 0 or len(q_s) > 0:
        
        if len(q_t) == 0:
            t2s = False
        elif len(q_s) == 0:
            t2s = True
            
        if t2s:
            unode = q_t.pop()
            
            res = r_t[unode] if unode in r_t else 0.0
#             if unode in p:
#                 p[unode] += res
#             else:
#                 p[unode] = res       
            r_t[unode] = 0.0
            
            for vnode in target_list[unode]:
                _val = (1 - alpha) * res / target_degree[unode]
                if vnode in r_s:
                    r_s[vnode] += _val
                else:
                    r_s[vnode] = _val

                res_vnode = r_s[vnode] if vnode in r_s else 0.0
                if res_vnode >= alpha_eps * source_degree[vnode]:
                    if vnode not in q_s:
                        q_s.append(vnode)
        else:
            unode = q_s.pop()
            
            res = r_s[unode] if unode in r_s else 0.0
            if unode in p:
                p[unode] += res
            else:
                p[unode] = res
            
            r_s[unode] = 0.0
            
            for vnode in source_list[unode]:
                _val = (1 - alpha) * res / source_degree[unode]
                if vnode in r_t:
                    r_t[vnode] += _val
                else:
                    r_t[vnode] = _val
                
                res_vnode = r_t[vnode] if vnode in r_t else 0.0
                if res_vnode >= alpha_eps * target_degree[vnode]:
                    if vnode not in q_t:
                        q_t.append(vnode)
    
    
#     p = sorted(p.items(), key = lambda k : k[1], reverse=True)
    return p

In [ ]:
edge_ppr_list = defaultdict( #target_type
                            lambda: defaultdict(  #source_type
                                lambda: defaultdict(  #relation_type
                                    lambda: defaultdict(  #target_id
                                        lambda: defaultdict( #source_id(
                                            lambda: float # ppr weight
                                        )))))

### cs alpha = 0.1 eplison = 0.001

### material science  alpha = 0.1 eplison = 0.0001 

In [ ]:
def sum_ppr():
    for target_type in graph.edge_list:
        te = graph.edge_list[target_type]
        for source_type in te:
            tes = te[source_type]
            for relation_type in tes:
                tesr = tes[relation_type]
                target_list = targetid2sourceidlist[target_type][source_type][relation_type]
                target_degree = targetid2degree[target_type][source_type][relation_type]
                source_list = targetid2sourceidlist[source_type][target_type]["rev_" + relation_type]
                source_degree = targetid2degree[source_type][target_type]["rev_" + relation_type]
                for target_id in tesr:
#                     ppr_val = calc_ppr_node(target_id, target_list, target_degree, source_list, source_degree, 0.25, 0.001)
                    ppr_val = calc_ppr_node(target_id, target_list, target_degree, source_list, source_degree, 0.1, 0.001)
                    for source_id in target_list[target_id]:
                        if source_id in ppr_val:
                            edge_ppr_list[target_type][source_type][relation_type][target_id][source_id] = ppr_val[source_id]
                        else:
                            edge_ppr_list[target_type][source_type][relation_type][target_id][source_id] = 0.0

In [ ]:
import time
t1 = time.time()
sum_ppr()
print(time.time() - t1)

In [ ]:
filter_edge_list = defaultdict( #target_type
                            lambda: defaultdict(  #source_type
                                lambda: defaultdict(  #relation_type
                                    lambda: defaultdict(  #target_id
                                        lambda: defaultdict( #source_id(
                                            lambda: int # time
                                        )))))

In [ ]:
print(dir(graph))

In [ ]:
graph.times

In [ ]:
pre_range   = {t: True for t in graph.times if t != None and t < 2014}

In [ ]:
pre_target_nodes   = []
for p_id, _time in enumerate(list(graph.node_feature['paper']['time'])):
    if _time in pre_range:
        pre_target_nodes += [p_id]

In [ ]:
for target_type in graph.edge_list:
    te = graph.edge_list[target_type]
    for source_type in te:
        tes = te[source_type]
        for relation_type in tes:
            tesr = tes[relation_type]
            if target_type == 'paper' and source_type == 'paper':
                for target_id in tesr:
                    tesrt = tesr[target_id]
                    sids = tesrt.keys()
                    
                    sid_w = dict()
                    for sid in sids:
                        if tesrt[sid] not in pre_range:
                            filter_edge_list[target_type][source_type][relation_type][target_id][sid] = tesrt[sid]
                        else:
                            sid_w[sid] = edge_ppr_list[target_type][source_type][relation_type][target_id][sid]
                    
                    n = len(sid_w)
                    if n <= 8:
                        for sid in sids:
                            filter_edge_list[target_type][source_type][relation_type][target_id][sid] = tesrt[sid]
                    else:
                        idxs = sorted(sid_w.items(), key=lambda k : k[1], reverse=True)
                        for sid, ppr_score in idxs[:8]:
                            filter_edge_list[target_type][source_type][relation_type][target_id][sid] = tesrt[sid]
            else:
                filter_edge_list[target_type][source_type][relation_type] = graph.edge_list[target_type][source_type][relation_type]

In [ ]:
graph.edge_list = filter_edge_list

In [ ]:
import dill
dill.dump(graph, open('dataset/graph_Engineering_ppr.pk', 'wb'))